## Test modules during dev


In [1]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [3]:
import sys
sys.path.append('../modules/')

In [8]:
%reload_ext autoreload
%autoreload 2
import datasets
import ML_classes

In [5]:
#dtree = datasets.read_filtered_datatree()
#dtree = datasets.calculate_magnitudes(dtree)

## Load data for different simulations and prepare it with right variables and non-dims.

In [5]:
DT = datasets.SimulationData(simulation_names=['P2L', 'DG'], filter_scales=['50','100','200','400'])

In [6]:
DT.preprocess_simulation_data(window_size=1)

In [12]:
DT.simulation_data

DataTree('None', parent=None)
├── DataTree('P2L')
│   ├── DataTree('50')
│   │       Dimensions:                      (Time: 7200, xh: 150, yh: 200, zl: 2, Xn: 1,
│   │                                         Yn: 1, zi: 3)
│   │       Coordinates:
│   │         * Time                         (Time) object 58kB 0001-01-11 00:00:00 ... 0...
│   │         * xh                           (xh) float64 1kB 4.0 12.0 ... 1.196e+03
│   │         * yh                           (yh) float64 2kB 4.0 12.0 ... 1.596e+03
│   │         * zl                           (zl) float64 16B 1.031e+03 1.033e+03
│   │         * zi                           (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables: (12/40)
│   │           dudx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dvdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dudy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dvdy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dhdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dhdy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           ...                           ...
│   │           dvdx_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dvdy_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dhdx_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           dhdy_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 3GB dask.array<chunksize=(36, 2, 200, 150, 1, 1), meta=np.ndarray>
│   │           uphp_rotated_nondim          (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
│   │           vphp_rotated_nondim          (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     50
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 50 km resolution
│   ├── DataTree('100')
│   │       Dimensions:                      (Time: 7200, xh: 60, yh: 80, zl: 2, Xn: 1,
│   │                                         Yn: 1, zi: 3)
│   │       Coordinates:
│   │         * Time                         (Time) object 58kB 0001-01-11 00:00:00 ... 0...
│   │         * xh                           (xh) float64 480B 10.0 30.0 ... 1.19e+03
│   │         * yh                           (yh) float64 640B 10.0 30.0 ... 1.59e+03
│   │         * zl                           (zl) float64 16B 1.031e+03 1.033e+03
│   │         * zi                           (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables: (12/40)
│   │           dudx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 553MB dask.array<chunksize=(36, 2, 80, 60, 1, 1), meta=np.ndarray>
│   │           dvdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 553MB dask.array<chunksize=(36, 2, 80, 60, 1, 1), meta=np.ndarray>
│   │           dudy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 553MB dask.array<chunksize=(36, 2, 80, 60, 1, 1), meta=np.ndarray>
│   │  

## ML data
Go from simulation data to batches. 
In this step things like dropping extra points, masking of data etc happen. 

In [23]:
ML_DT_train = datasets.MLDataset(simulation_data=DT,input_variables=['dudx_widened','filter_scale'],
                                 output_variables=['uphp'], time_range=slice(-50, None))
ML_DT_test = datasets.MLDataset(simulation_data=DT,input_variables=['dudx_widened','filter_scale'],
                                 output_variables=['uphp'], time_range=slice(-100, -75))


In [28]:
ML_DT_train.input_variables

['dudx_widened', 'filter_scale']

In [24]:
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0034 seconds
Time subsampling took: 0.0122 seconds
Horizontal subsampling took: 0.0105 seconds
h_mask_ml_variables took: 0.0509 seconds
stack_physical_dimensions took: 0.0388 seconds
will load upto: 0.0249 gb
load took: 1.6290 seconds
drop_nans took: 0.0190 seconds
Seed set as:42
randomize_along_points took: 0.0190 seconds
Picked: 30000points
pick_uniform_points took: 0.0058 seconds
concat_datatree_nodes took: 0.0152 seconds
randomize_concatenated_ml_dataset took: 0.0090 seconds
generate_batches took: 0.0002 seconds


In [25]:
ML_DT_test.create_ML_variables()

choose_ml_variables took: 0.0036 seconds
Time subsampling took: 0.0121 seconds
Horizontal subsampling took: 0.0100 seconds
h_mask_ml_variables took: 0.0569 seconds
stack_physical_dimensions took: 0.0354 seconds
will load upto: 0.01245 gb
load took: 1.8535 seconds
drop_nans took: 0.0148 seconds
Seed set as:42
randomize_along_points took: 0.0133 seconds
Picked: 15000points
pick_uniform_points took: 0.0057 seconds
concat_datatree_nodes took: 0.0129 seconds
randomize_concatenated_ml_dataset took: 0.0044 seconds
generate_batches took: 0.0002 seconds


In [26]:
ML_DT_train.ml_dataset

DataTree('None', parent=None)
├── DataTree('P2L')
│   ├── DataTree('50')
│   │       Dimensions:       (Xn: 1, Yn: 1, points: 30000)
│   │       Coordinates:
│   │         * points        (points) object 240kB MultiIndex
│   │         * Time          (points) object 240kB 0197-05-02 00:00:00 ... 0197-04-12 00...
│   │         * xh            (points) float64 240kB 772.0 900.0 4.0 ... 1.156e+03 772.0
│   │         * yh            (points) float64 240kB 1.028e+03 1.54e+03 ... 1.54e+03 132.0
│   │         * zl            (points) float64 240kB 1.033e+03 1.033e+03 ... 1.033e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables:
│   │           dudx_widened  (Xn, Yn, points) float64 240kB 8.225e-07 ... -1.833e-07
│   │           filter_scale  (points) float64 240kB 5e+04 5e+04 5e+04 ... 5e+04 5e+04 5e+04
│   │           uphp          (points) float32 120kB 0.1546 0.06586 ... -0.124 0.01065
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     50
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 50 km resolution
│   ├── DataTree('100')
│   │       Dimensions:       (Xn: 1, Yn: 1, points: 30000)
│   │       Coordinates:
│   │         * points        (points) object 240kB MultiIndex
│   │         * Time          (points) object 240kB 0196-11-23 00:00:00 ... 0196-12-23 00...
│   │         * xh            (points) float64 240kB 570.0 250.0 1.05e+03 ... 1.05e+03 410.0
│   │         * yh            (points) float64 240kB 1.29e+03 650.0 1.13e+03 ... 90.0 250.0
│   │         * zl            (points) float64 240kB 1.033e+03 1.033e+03 ... 1.033e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables:
│   │           dudx_widened  (Xn, Yn, points) float64 240kB 1.583e-08 ... 2.565e-07
│   │           filter_scale  (points) float64 240kB 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
│   │           uphp          (points) float32 120kB 0.00746 -0.2561 ... -0.0005843 -0.0261
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     100
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 100 km resolution
│   ├── DataTree('200')
│   │       Dimensions:       (Xn: 1, Yn: 1, points: 30000)
│   │       Coordinates:
│   │         * points        (points) object 240kB MultiIndex
│   │         * Time          (points) object 240kB 0197-07-31 00:00:00 ... 0197-08-30 00...
│   │         * xh            (points) float64 240kB 900.0 820.0 580.0 ... 1.06e+03 500.0
│   │         * yh            (points) float64 240kB 1.38e+03 100.0 740.0 ... 1.38e+03 20.0
│   │         * zl            (points) float64 240kB 1.033e+03 1.033e+03 ... 1.031e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables:
│   │           dudx_widened  (Xn, Yn, points) float64 240kB -2.412e-07 ... -2.055e-08
│   │           filter_scale  (points) float64 240kB 2e+05 2e+05 2e+05 ... 2e+05 2e+05 2e+05
│   │           uphp          (points) float32 120kB -0.1042 -0.004129 ... 0.2199 0.01214
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     200
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 200 km resolution
│   └── DataTree('400')
│           Dimensions:       (Xn: 1, Yn: 1, points: 30000)
│           Coordinates:
│             * points        (points) object 240kB MultiIndex
│             * Time          (points) object 240kB 0198-02-16 00:00:00 ... 0197-10-19 00...
│             * xh            (points) float64 240kB 200.0 920.0 280.0 ... 120.0 360.0
│             * yh            (points) float64 240kB 1.56e+03 360.0 ... 1.56e+03 1.4e+03
│             * zl            (points) fl

In [21]:
len(ML_DT_train.ml_batches)

100

In [22]:
ML_DT_train.ml_batches[0]

<xarray.Dataset> Size: 144kB
Dimensions:       (Xn: 1, Yn: 1, points: 2400)
Coordinates:
  * points        (points) object 19kB MultiIndex
  * Time          (points) object 19kB 0197-10-29 00:00:00 ... 0098-07-31 00:...
  * xh            (points) float64 19kB 1.14e+03 324.0 760.0 ... 1.028e+03 7.26
  * yh            (points) float64 19kB 340.0 772.0 1.24e+03 ... 1.22e+03 44.73
  * zl            (points) float64 19kB 1.033e+03 1.033e+03 ... 1.035e+03
Dimensions without coordinates: Xn, Yn
Data variables:
    dudx_widened  (Xn, Yn, points) float64 19kB 9.201e-08 ... 4.327e-08
    filter_scale  (points) float64 19kB 2e+05 5e+04 4e+05 ... 4e+05 5e+04 2e+05
    uphp          (points) float32 10kB -0.1388 -1.484 2.703 ... 16.24 0.7279
Attributes:
    simulation_name:  P2L
    filter_scale:     50
    source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
    description:      Dataset for P2L experiment at 50 km resolution

## The ML part


In [21]:
ANN_model = ML_classes.ANN(num_in = 7)

In [23]:
ANN_model = ML_classes.PointwiseANN(num_in = 7)

In [24]:
ANN_model.count_parameters()

842


In [19]:
ANN_model.count_parameters()

842


In [35]:
ANN_model

In [33]:
regress_sys = ML_classes.RegressionSystem(ANN_model, local_norm=True)

In [34]:
%%time
regress_sys.train_system(ML_DT_train, 31, 1)

AttributeError: 'MLDataset' object has no attribute 'input_channels'